In [1]:
import angr, claripy
from pwn import *

In [2]:
elf = ELF("./gates")

[*] '/home/kali/machines/ctf/wani/rev/gates/loot/gates'
    Arch:     amd64-64-little
    RELRO:    Full RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      PIE enabled


INFO     | 2024-06-21 20:27:55,730 | pwnlib.elf.elf | '/home/kali/machines/ctf/wani/rev/gates/loot/gates'
Arch:     amd64-64-little
RELRO:    Full RELRO
Stack:    No canary found
NX:       NX enabled
PIE:      PIE enabled


In [3]:
!objdump -d ./gates


./gates:     file format elf64-x86-64


Disassembly of section .init:

0000000000001000 <.init>:
    1000:	f3 0f 1e fa          	endbr64
    1004:	48 83 ec 08          	sub    $0x8,%rsp
    1008:	48 8b 05 d9 2f 00 00 	mov    0x2fd9(%rip),%rax        # 3fe8 <getc@plt+0x2f78>
    100f:	48 85 c0             	test   %rax,%rax
    1012:	74 02                	je     1016 <__cxa_finalize@plt-0x3a>
    1014:	ff d0                	call   *%rax
    1016:	48 83 c4 08          	add    $0x8,%rsp
    101a:	c3                   	ret

Disassembly of section .plt:

0000000000001020 <.plt>:
    1020:	ff 35 92 2f 00 00    	push   0x2f92(%rip)        # 3fb8 <getc@plt+0x2f48>
    1026:	f2 ff 25 93 2f 00 00 	bnd jmp *0x2f93(%rip)        # 3fc0 <getc@plt+0x2f50>
    102d:	0f 1f 00             	nopl   (%rax)
    1030:	f3 0f 1e fa          	endbr64
    1034:	68 00 00 00 00       	push   $0x0
    1039:	f2 e9 e1 ff ff ff    	bnd jmp 1020 <__cxa_finalize@plt-0x30>
    103f:	90                   	nop
    1040:	f3

In [4]:
p = angr.Project("./gates", main_opts={"base_addr": 0x100000}, auto_load_libs=False)

In [5]:
s = p.factory.entry_state(addr=0x1010bc)

vs = [claripy.BVS(f"chr{x}", 8) for x in range(32)]

for i, v in enumerate(vs):
    s.memory.store(0x10404c + (i * 0x10), 1, 1)
    s.memory.store(0x10404c + 1 + (i * 0x10), v, 1)

sm = p.factory.simgr(s, veritesting=True)

In [6]:
results = sm.explore(find=lambda s: b"Correct" in s.posix.dumps(1), avoid=lambda s: b"Wrong" in s.posix.dumps(1))

WARNING  | 2024-06-21 20:27:55,979 | angr.storage.memory_mixins.default_filler_mixin | The program is accessing register with an unspecified value. This could indicate unwanted behavior.
WARNING  | 2024-06-21 20:27:55,979 | angr.storage.memory_mixins.default_filler_mixin | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING  | 2024-06-21 20:27:55,980 | angr.storage.memory_mixins.default_filler_mixin | 1) setting a value to the initial state
WARNING  | 2024-06-21 20:27:55,980 | angr.storage.memory_mixins.default_filler_mixin | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING  | 2024-06-21 20:27:55,981 | angr.storage.memory_mixins.default_filler_mixin | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to suppress these messages.
WARNING  | 2024-06-21 20:27:55,982 | angr.storage.memory_mixins.default_filler_mixin | Filling register 

In [7]:
results

<SimulationManager with 1 found, 1 avoid>

In [9]:
results.avoid[0].solver.eval(vs[2])

65

In [15]:
for i in range(32):
    print(f"{chr(results.found[0].solver.eval(vs[i]))}", end="")

FLAG{INTr0dUction_70_R3v3R$1NG1}